In [2]:
# imports
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine

# snowflake config
## connection
SF_USER = "LJOSEPH"
SF_PASSWORD = "<>"
ACCOUNT = "zg83245.east-us-2.azure"
DATABASE = "INGEST_DEV"
SCHEMA = "AGG_BRIDGE"
WAREHOUSE = "INGEST_DEV_WH"

/usr/local/Caskroom/miniconda/base/envs/personal-notebook/lib/python3.5/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 3.5 support will be dropped in the next release of cryptography. Please upgrade your Python.
  from cryptography import x509


In [3]:
# connect to Snowflake
engine = create_engine(
    # 'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
    'snowflake://{user}:{password}@{account}'.format(
        user=SF_USER,
        password=SF_PASSWORD,
        account=ACCOUNT,
#         database=DATABASE,
#         schema=SCHEMA,
#         warehouse=WAREHOUSE
    )
)

cnxn = engine.connect()

In [4]:
QUERY = """
    With ALL_JPAR_AGENTS as (
    
        Select
            Agent.Member_State_ID as Agent_Member_State_ID
            
        FROM
            "INGEST_DEV"."REF_DATA"."TREC_CURRENT_TAB" as Agent
            
        LEFT JOIN
            "INGEST_DEV"."REF_DATA"."TREC_CURRENT_TAB" as Broker
            ON Agent.RELATED_LICENSE_STATE_ID = Broker.MEMBER_STATE_ID
            
        WHERE 
            Agent.License_Status in ('Current and Active','Probation and Active') 
            AND Broker.Member_State_ID in 
                (
                'TX|09002729', -- JPAR Dallas - National HQ + DFW)
                'TX|09008344' -- JPAR The Sears Group (Houston Franchise)
                    )
    ),

    ALL_JPAR_AGENT_IDS as (
        SELECT
            Agent_Member_State_ID
        FROM
            ALL_JPAR_AGENTS
    ),

    ORDERED_JPAR_BUY_SIDE_LISTINGS as (

        SELECT 
            ROW_NUMBER() OVER (PARTITION BY NormalizedAddress ORDER BY close_date desc) RN,
            NormalizedAddress,
            listing_fact_id,
            listing_guid   
        FROM
            "DW_DEV"."MAIN"."LISTING_FACT" 
        WHERE
            BUYER_AGENT_STATE_ID in (SELECT * FROM ALL_JPAR_AGENT_IDS)
            OR CO_BUYER_AGENT_STATE_ID in (SELECT * FROM ALL_JPAR_AGENT_IDS)

  )

  SELECT * FROM ORDERED_JPAR_BUY_SIDE_LISTINGS WHERE RN = 1

"""

In [5]:
df = pd.read_sql(
    QUERY,
    con=engine
)

In [6]:
print(len(df))

36734
